In [52]:
import sys
import random
import numpy as np
from numpy import array,zeros,dot
from numpy import genfromtxt

In [2]:
data = genfromtxt('mnist_train.csv', delimiter=',')

In [7]:
a = array([[1,2,3,4,5],[11,12,13,14,15],[11,121,131,141,151]])

In [96]:
class NN:
    def __init__(self, inp_unit, out_unit,  hidden_shape, activation, itr=1000, alpha=0.01, momentum=0.9, decay=0.1):
        self.inp_unit,self.out_unit,self.hidden_shape,self.itr,self.alpha,self.momentum,self.decay,self.activation = inp_unit,\
                                                            out_unit, hidden_shape, itr, alpha, momentum, decay, activation 
        self.flag,self.W,self.B,self.arc_dim = True,0,0,0     
        self.pred,self.h_layer,self.h_layer_A,self.temp,self.Frist_time,self._v,self._m = 0,0,0,0,0,0,0
        self.testing = []
    
    def initz(self,inp_unit,out_unit):
        self.W,self.B,self.arc_dim = 0,0,0
        self.pred,self.temp,self.Frist_time,self._v,self._m = 0,0,True,zeros((len(self.hidden_shape)+1))\
                                                                            ,zeros((len(self.hidden_shape)+1))
        if len(inp_unit.shape) > 1 and len(out_unit.shape) > 1:
            self.W,self.B,self.arc_dim = [],[],[inp_unit.shape[1]] + self.hidden_shape + [out_unit.shape[1]]
        else:
            self.flag = False
    
    def sigmoid(self,x,flag):
        print("sig")
        x[x < -100] = -100
        return 1 / (1.0 + np.exp(-x)) if flag == 0 else np.exp(-x)/((1+np.exp(-x))**2)

    def tanh(self,x,flag):
        print("tanh")
        x[x < -100] = -100
        return np.tanh(x) if flag == 0 else 1.0 - np.tanh(x)**2.0 

    def random_weight(self,shp):
        return np.random.normal(loc = 0, scale = 0.3, size = shp)
        
    def _feed_forward(self,A):
        self.h_layer,self.h_layer_A = [],[]
        if self.Frist_time is True:
            for idx in range(len(self.arc_dim)-1):
                self.W.append(self.random_weight([self.arc_dim[idx],self.arc_dim[idx+1]]))
                self.B.append(self.random_weight([1,self.arc_dim[idx+1]]))
            self.Frist_time = False
        for idx in range(len(self.arc_dim)-2):
            if self.activation[idx] is "sigmoid":
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.sigmoid(A,0)
                self.h_layer_A.append(C)
            else:
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.tanh(A,0)
                self.h_layer_A.append(C)
        
        if self.activation[-1] is "sigmoid":
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.sigmoid(self.temp,0) 
        else:
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.tanh(self.temp,0)
    
    def cost_func(self,targets,predct):
        return(0.5*sum((self.out_unit-self.pred)**2))
    
    def _back_prop(self,Inp):
        weight_delta = []
        if self.activation[-1] == "sigmoid":
            delta = np.multiply(-(self.out_unit-self.pred), self.sigmoid(self.temp,1))
        else:
            delta = np.multiply(-(self.out_unit-self.pred), self.tanh(self.temp,1))
        weight_delta.append(np.dot(self.h_layer_A[-1].T, delta))
        for idx in reversed(range(len(self.h_layer))):
            if self.activation[idx] == "sigmoid":
                delta2 = np.dot(delta, self.W[idx+1].T)*self.sigmoid(self.h_layer[idx],1)
            else:
                delta2 = np.dot(delta, self.W[idx+1].T)*self.tanh(self.h_layer[idx],1)
            if idx > 0:
                weight_delta.append(np.dot(self.h_layer_A[idx-1].T, delta2))
            else:
                weight_delta.append(np.dot(Inp.T, delta2))
            delta = delta2        
        weight_delta.reverse()
        return weight_delta
    
    def _weight_update_SG(self,dx,alpha):
        for i in range(len(dx)):
            self.W[i] += alpha * dx[i]
    
    def _weight_update_Momentum(self,dx,alpha,mu):
        for i in range(len(dx)):
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += self._v[i]
            
    def _weight_update_Nag(self,dx,alpha,mu):
        for i in range(len(dx)):
            v_prev = self._v[i]
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += -mu * v_prev + (1 + mu) * self._v[i]
            
    def _weight_update_Adam(self,dx,t,alpha,eps=1e-8,beta1=0.9,beta2=0.999):
        for i in range(len(dx)):
            self._m[i] = beta1*self._m[i] + (1-beta1)*dx[i]
            mt = self._m[i] / (1-beta1**t)
            self._v[i] = beta2*self._v[i] + (1-beta2)*(dx[i]**2)
            vt = self._v[i] / (1-beta2**t)
            self.W[i] += - alpha * mt / (np.sqrt(vt) + eps)
            
    def train(self):
        self.initz(self.inp_unit,self.out_unit)
        a = []
        for i in range(1):
            self._feed_forward(self.inp_unit)
            dx = self._back_prop(self.inp_unit)
            self._weight_update_SG(dx,self.alpha)
            if(i%10 == 0):
                print(self.cost_func(self.out_unit,self.pred))

In [98]:
# nn = NN(a,array([[1,0],[0,1],[1,0]]),[3,4,3],['tanh','sigmoid','tanh','sigmoid'])['sigmoid','sigmoid','sigmoid','sigmoid']
nn = NN(train_x[:100],train_y_1[:100],[10,15,20],['tanh','sigmoid','tanh','sigmoid'])
nn.train()
# nn.W


tanh
sig
tanh
sig
sig
tanh
sig
tanh
[  9.93136044  24.34456002  15.65693489  13.84249636  31.44655095
  11.87921267  16.12625758  11.17678312   8.89278023  18.52630415]


In [81]:
((nn.out_unit-nn.pred)**2)

array([[ 0.12357297,  0.19171963,  0.77030515,  0.35299794,  0.12736308,
         0.33802354,  0.73868581,  0.0971314 ,  0.06850916,  0.22215888],
       [ 0.08656557,  0.36946421,  0.89671613,  0.49490158,  0.2051064 ,
         0.03785851,  0.68891856,  0.14839919,  0.07309702,  0.3440195 ],
       [ 0.44282118,  0.36648314,  0.76367139,  0.35637724,  0.17987684,
         0.13263386,  0.83124579,  0.1042941 ,  0.32212354,  0.63859308],
       [ 0.34933181,  0.04858795,  0.60939058,  0.31904222,  0.39012747,
         0.01800612,  0.70426096,  0.13102846,  0.47693591,  0.75947348],
       [ 0.05089318,  0.41369365,  0.46866828,  0.16577867,  0.30768049,
         0.09305468,  0.61438687,  0.1387033 ,  0.15917553,  0.16490309],
       [ 0.09062592,  0.25380053,  0.00348857,  0.35584156,  0.19882528,
         0.16797359,  0.60413153,  0.06574701,  0.07350728,  0.18168819],
       [ 0.09485777,  0.24598157,  0.63220512,  0.12625223,  0.20166974,
         0.28098117,  0.63685606,  0.12879504

In [8]:
nn.testing[7]

array([[  0.00000000e+000,   0.00000000e+000,   7.56979037e-145,
          2.31432783e-292,   0.00000000e+000,               nan,
          0.00000000e+000,   7.34369582e-058,   1.67797068e-082,
          0.00000000e+000],
       [  4.33779636e-295,   0.00000000e+000,   1.22890822e-040,
          4.59832562e-258,   0.00000000e+000,               nan,
          3.83041926e-256,   0.00000000e+000,   7.40152730e-208,
                      nan],
       [  4.36259965e-321,   4.79224911e-032,   2.70085820e-011,
          0.00000000e+000,   2.06923903e-153,   0.00000000e+000,
          3.78476751e-104,   1.18032283e-320,   1.49050717e-204,
          0.00000000e+000],
       [  0.00000000e+000,   1.41907957e-078,   1.92257079e-085,
          1.03807570e-004,   2.60895729e-155,               nan,
          0.00000000e+000,   0.00000000e+000,   9.72904008e-113,
          1.44861074e-010],
       [  3.50366521e-027,               nan,   6.96161857e-044,
          1.04691651e-160,   7.83854772e-07

In [48]:
# a = -300
# print((np.exp(-a)/((1+np.exp(-a))**2)))
a = -100
print((np.exp(-a)/((1+np.exp(-a))**2)))

3.72007597602e-44


In [30]:
# np.finfo(a.dtype)
print(np.finfo(a).maxexp,np.finfo(a).minexp)

1024 -1022


In [53]:
a = array([[1,2,3,4,5,6],[1,2,3,4,6,5]])
a[a < 3] = 0
a

array([[0, 0, 3, 4, 5, 6],
       [0, 0, 3, 4, 6, 5]])

In [69]:
print(nn.W[0].shape,
nn.W[1].shape,
nn.W[2].shape,
nn.W[6].shape)

IndexError: list index out of range

In [3]:
b = array([[1,0],[0,1],[1,0]])

In [106]:
q = array([[1,2,3],[4,5,6]])
q = q + array([1])

# Preprocessing

In [3]:
train = data[:50000]
valid = data[50000:]

In [4]:
train_y = train[:,0]
train_x = train[:,1:]
train_y_1 = np.eye(10)[train_y.astype(int)]
valid_y = valid[:,0]
valid_x = valid[:,1:]
valid_y_1 = np.eye(10)[valid_y.astype(int)]

In [213]:
import pickle

In [216]:
pickle.dump(train_x,open("train_x.pkl",'wb'))
pickle.dump(train_y_1,open("train_y.pkl",'wb'))
pickle.dump(valid_x,open("valid_x.pkl",'wb'))
pickle.dump(valid_y_1,open("valid_y.pkl",'wb'))

# Testing Tensorflow

In [133]:
import tensorflow as tf

In [163]:
x = tf.placeholder(tf.float32, [None, 5])
y = tf.placeholder(tf.float32, [None, 2])
w1 = tf.placeholder(tf.float32, [5, 3])
w2 = tf.placeholder(tf.float32, [3, 4])
w3 = tf.placeholder(tf.float32, [4, 5])
w4 = tf.placeholder(tf.float32, [5, 2])
b1 = tf.placeholder(tf.float32, [1, 3])
b2 = tf.placeholder(tf.float32, [1, 4])
b3 = tf.placeholder(tf.float32, [1, 5])
b4 = tf.placeholder(tf.float32, [1, 2])

In [164]:
g1 = tf.tanh(tf.add(tf.matmul(x, w1), b1))
print(g1)
g2 = tf.sigmoid(tf.add(tf.matmul(g1, w2), b2))
print(g2)
g3 = tf.tanh(tf.add(tf.matmul(g2, w3), b3))
print(g3)
g4 = (tf.add(tf.matmul(g3, w4), b4))
print(g4)

Tensor("Tanh_17:0", shape=(?, 3), dtype=float32)
Tensor("Sigmoid_9:0", shape=(?, 4), dtype=float32)
Tensor("Tanh_18:0", shape=(?, 5), dtype=float32)
Tensor("Add_35:0", shape=(?, 2), dtype=float32)


In [165]:
with tf.Session() as sess:
    f = sess.run(g4,feed_dict = {x: a, y: b, w1: nn.W[0], w2: nn.W[1], w3: nn.W[2], w4: nn.W[3],
                             b1: nn.B[0], b2: nn.B[1], b3: nn.B[2], b4: nn.B[3]})

In [166]:
f

array([[-0.12157428, -0.07123873],
       [-0.11776307, -0.07139242],
       [-0.13612098, -0.07489069]], dtype=float32)

In [8]:
array([[1,2],[2,3]]) - array([[3,1],[1,2]])

array([[-2,  1],
       [ 1,  1]])